##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Aprendizado por transferência com o TensorFlow Hub

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/images/transfer_learning_with_hub.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
  <td>     <a href="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo do TF Hub</a>
</td>
</table>

O [TensorFlow Hub](https://tfhub.dev/) é um repositório de modelos pré-treinados do TensorFlow.

Este tutorial demonstra como:

1. Usar modelos do TensorFlow Hub com `tf.keras`.
2. Usar um modelo de classificação de imagens do TensorFlow Hub.
3. Fazer um aprendizado por transferência simples a fim de realizar o ajuste fino de um modelo para suas próprias classes de imagens.

## Configuração

In [ ]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

%load_ext tensorboard

## Classificador do ImageNet

Você começará usando um modelo classificador pré-trienado com o dataset de referência do [ImageNet](https://en.wikipedia.org/wiki/ImageNet). Não é necessário fazer nenhum treinamento inicial!

### Baixar o classificador

Selecione um modelo <a href="https://arxiv.org/abs/1801.04381" class="external">MobileNetV2</a> pré-treinado [do TensorFlow Hub](https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2) e encapsule-o como uma camada do Keras com[`hub.KerasLayer`](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer). Qualquer <a href="https://tfhub.dev/s?q=tf2&amp;module-type=image-classification/" class="external">modelo classificador de imagens compatível</a> do TensorFlow Hub funcionará neste tutorial, incluindo os exemplos fornecidos no menu suspenso abaixo.

In [ ]:
mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
inception_v3 = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"

classifier_model = mobilenet_v2 #@param ["mobilenet_v2", "inception_v3"] {type:"raw"}

In [ ]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))
])

### Executar em uma única imagem

Baixe uma única imagem para testar o modelo nela:

In [ ]:
grace_hopper = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = Image.open(grace_hopper).resize(IMAGE_SHAPE)
grace_hopper

In [ ]:
grace_hopper = np.array(grace_hopper)/255.0
grace_hopper.shape

Adicione uma dimensão de lote (com `np.newaxis`) e passe a imagem para o modelo:

In [ ]:
result = classifier.predict(grace_hopper[np.newaxis, ...])
result.shape

O resultado é um vetor de logits com 1.001 elementos classificando a probabilidade de cada classe para a imagem.

O ID da classe superior pode ser encontrado com `tf.math.argmax`:

In [ ]:
predicted_class = tf.math.argmax(result[0], axis=-1)
predicted_class

### Decodificar as previsões

Pegue o ID de `predicted_class` (como `653`) e busque os rótulos do dataset do ImageNet para decodificar as previsões:

In [ ]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [ ]:
plt.imshow(grace_hopper)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())

## Aprendizado por transferência simples

Mas e se você quiser criar um classificador personalizado usando seu próprio dataset que tenha classes não incluídas no dataset original do ImageNet (usado para treinar o modelo pré-treinado)?

Para fazer isso, você pode:

1. Selecionar um modelo pré-treinado do TensorFlow Hub; e
2. Treinar novamente a camada superior (última) para reconhecer as classes do seu dataset personalizado.

### Dataset

Neste exemplo, você usará o dataset Flowers do TensorFlow:

In [ ]:
import pathlib

data_file = tf.keras.utils.get_file(
  'flower_photos.tgz',
  'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
  cache_dir='.',
   extract=True)

data_root = pathlib.Path(data_file).with_suffix('')

Primeiro, carregue os dados no modelo usando os dados de imagens para fora do disco com `tf.keras.utils.image_dataset_from_directory`, e será gerado um `tf.data.Dataset`:

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

O dataset de flores tem cinco classes:

In [ ]:
class_names = np.array(train_ds.class_names)
print(class_names)

Segundo, como o TensorFlow Hub para modelos de imagens espera entradas float no intervalo `[0, 1]`, use a camada de pré-processamento `tf.keras.layers.Rescaling` para alcançar esse objetivo.

Observação: você também pode incluir a camada `tf.keras.layers.Rescaling` dentro do modelo. Confira uma discussão sobre as contrapartidas no guia [Trabalhando com camadas de pré-processamento](https://www.tensorflow.org/guide/keras/preprocessing_layers).

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

Terceiro, finalize o pipeline de dados usando pré-busca em buffer com `Dataset.prefetch` para poder gerar os dados a partir do disco sem causar problemas de bloqueio de I/O.

Esses são alguns dos métodos de `tf.data` mais importantes que você deve usar ao carregar dados. Quem tiver interesse pode aprender mais sobre isso, além de como fazer cache de dados no disco e outras técnicas, no guia [Melhor desempenho com a API tf.data](https://www.tensorflow.org/guide/data_performance#prefetching).

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

### Executar o classificador em um lote de imagens

Agora, execute o classificador em um lote de imagens:

In [ ]:
result_batch = classifier.predict(train_ds)

In [ ]:
predicted_class_names = imagenet_labels[tf.math.argmax(result_batch, axis=-1)]
predicted_class_names

Confira como essas previsões se alinham com as imagens:

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("ImageNet predictions")

Observação: todas as imagens têm uma licença CC-BY. Os criadores são listados no arquivo LICENSE.txt.

Os resultados estão longe da perfeição, mas são razoáveis, considerando que o modelo não foi treinado para essas classes (exceto por "daisy" – margarida).

### Baixar o modelo sem cabeçalho

O TensorFlow Hub também distribui modelos sem a camada de classificação superior. Eles podem ser usados para fazer o aprendizado por transferência com facilidade.

Selecione um modelo <a href="https://arxiv.org/abs/1801.04381" class="external">MobileNetV2</a> pré-treinado <a href="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4" class="external">do TensorFlow Hub</a>. Qualquer <a href="https://tfhub.dev/s?module-type=image-feature-vector&amp;q=tf2" class="external">modelo de vetor de características de imagens compatível</a> do TensorFlow Hub funcionária neste tutorial, incluindo os exemplos do menu suspenso abaixo.

In [ ]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

feature_extractor_model = mobilenet_v2 #@param ["mobilenet_v2", "inception_v3"] {type:"raw"}

Crie o extrator de características encapsulando o modelo pré-treinado como uma camada do Keras com [`hub.KerasLayer`](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer). Use o argumento `trainable=False` para congelar as variáveis; dessa forma, o treinamento modifica somente a camada do novo classificador:

In [ ]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

O extrator de características retorna um vetor de tamanho 1.280 para cada imagem (o tamanho do lote de imagens permanece como 32 neste exemplo):

In [ ]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

### Anexar um cabeçalho de classificação

Para completar o modelo, encapsule a camada de extração de características em um modelo do `tf.keras.Sequential` e adicione uma camada totalmente conectada para a classificação:

In [ ]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

In [ ]:
predictions = model(image_batch)

In [ ]:
predictions.shape

### Treinar o modelo

Use `Model.compile` para configurar o processo de treinamento e adicione um callback `tf.keras.callbacks.TensorBoard` para criar e armazenar logs:

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

Agora, use o método `Model.fit` para treinar o modelo.

Para manter este exemplo curto, você fará o treinamento com 10 épocas. Para visualizar o progresso do treinamento no TensorBoard posteriormente, crie e armazene logs em um [callback do TensorBoard](https://www.tensorflow.org/tensorboard/get_started#using_tensorboard_with_keras_modelfit).

In [ ]:
NUM_EPOCHS = 10

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Inicie o TensorBoard para ver como as métricas mudam a cada época e para monitorar outros valores de escalares:

In [ ]:
%tensorboard --logdir logs/fit

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/images/tensorboard_transfer_learning_with_hub.png?raw=1"/> -->

### Verificar as previsões

Obtenha a lista ordenada dos nomes das classes a partir das previsões do modelo:

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_id = tf.math.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]
print(predicted_label_batch)

Plote as previsões do modelo:

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)

for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")

## Exportar e carregar seu modelo

Agora que você treinou o modelo, exporte-o como um SavedModel para reutilizá-lo posteriormente.

In [ ]:
t = time.time()

export_path = "/tmp/saved_models/{}".format(int(t))
model.save(export_path)

export_path

Confirme se você consegue recarregar o SavedModel e se o modelo consegue gerar os mesmos resultados:

In [ ]:
reloaded = tf.keras.models.load_model(export_path)

In [ ]:
result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

In [ ]:
abs(reloaded_result_batch - result_batch).max()

In [ ]:
reloaded_predicted_id = tf.math.argmax(reloaded_result_batch, axis=-1)
reloaded_predicted_label_batch = class_names[reloaded_predicted_id]
print(reloaded_predicted_label_batch)

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(reloaded_predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")

## Próximos passos

Você pode usar o SavedModel para carregá-lo para fazer inferência ou convertê-lo em um modelo do [TensorFlow Lite](https://www.tensorflow.org/lite/models/convert/) (para aprendizado de máquina em dispositivos) ou em um modelo do [TensorFlow.js](https://www.tensorflow.org/js/tutorials#convert_pretrained_models_to_tensorflowjs) (para aprendizado de máquina no JavaScript).

Confira [mais tutoriais](https://www.tensorflow.org/hub/tutorials) para aprender a usar modelos pré-treinados do TensorFlow com tarefas de imagens, textos, áudios e vídeos.